In [1]:
import pandas as pd

In [2]:
babylon = pd.read_csv('babylonbee_urls.csv', quotechar="\"")

In [3]:
onion = pd.read_csv('onion_urls_brief.csv')

In [4]:
onion = onion.drop_duplicates()

In [5]:
spoof_2020 = pd.read_csv('spoof_urls_2020.csv')

In [6]:
spoof_2019 = pd.read_csv('spoof_urls_2019.csv')

In [7]:
spoof_2021 = pd.read_csv('spoof_urls_2021.csv')

In [8]:
satire = pd.concat([babylon, onion, spoof_2020, spoof_2021, spoof_2019])

In [9]:
len(satire)

20146

In [39]:
wel = pd.read_csv('../data/WELFake_Dataset.csv')

In [40]:
wel = wel.dropna()

In [41]:
fake_wel = wel[wel['label'] == 1]
fake_wel.drop_duplicates(inplace=True)
real_wel = wel[wel['label'] == 0]
real_wel.drop_duplicates(inplace=True)
satire.drop_duplicates(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [42]:
real_wel.rename(columns={'title' : 'Heading', 'text' : 'Body'}, inplace=True)
fake_wel.rename(columns={'title' : 'Heading', 'text' : 'Body'}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [43]:
real_wel.dropna(inplace=True)
fake_wel.dropna(inplace=True)
satire.dropna(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
import nltk

In [45]:
fake_wel['token_heading'] = fake_wel['Heading'].apply(lambda row: nltk.word_tokenize(row))
real_wel['token_heading'] = real_wel['Heading'].apply(lambda row: nltk.word_tokenize(row))
satire['token_heading'] = satire['Heading'].apply(lambda row: nltk.word_tokenize(row))

fake_wel['token_body'] = fake_wel['Body'].apply(lambda row: nltk.word_tokenize(row))
real_wel['token_body'] = real_wel['Body'].apply(lambda row: nltk.word_tokenize(row))
satire['token_body'] = satire['Body'].apply(lambda row: nltk.word_tokenize(row))

fake_wel['body_len'] = fake_wel['token_body'].apply(len)
real_wel['body_len'] = real_wel['token_body'].apply(len)
satire['body_len'] = satire['token_body'].apply(len)

fake_wel['heading_len'] = fake_wel['token_heading'].apply(len)
real_wel['heading_len'] = real_wel['token_heading'].apply(len)
satire['heading_len'] = satire['token_heading'].apply(len)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [46]:
satire = satire[satire['body_len'] > 0]
fake_wel = fake_wel[fake_wel['body_len'] > 0]
real_wel = real_wel[real_wel['body_len'] > 0]

satire = satire[satire['heading_len'] > 0]
fake_wel = fake_wel[fake_wel['heading_len'] > 0]
real_wel = real_wel[real_wel['heading_len'] > 0]

In [47]:
satire['label'] = 'satire'

In [48]:
real_wel['label'] = 'real'

In [49]:
fake_wel['label'] = 'fake'

In [50]:
import fasttext
PRETRAINED_MODEL_PATH = 'lid.176.ftz'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)
def find_lang(t):
    clean = t.replace("\n", " ")
    return model.predict(clean)[0][0]
satire['language'] = satire['Body'].apply(lambda x: find_lang(x))
real_wel['language'] = real_wel['Body'].apply(lambda x: find_lang(x))
fake_wel['language'] = fake_wel['Body'].apply(lambda x: find_lang(x))

In [51]:
satire = satire[satire['language'] == '__label__en']
real_wel = real_wel[real_wel['language'] == '__label__en']
fake_wel = fake_wel[fake_wel['language'] == '__label__en']

In [52]:
satire_final = satire[['Heading', 'Body', 'label']]
satire_final.dropna(inplace=True)

fake_wel = fake_wel.sample(n=len(satire), random_state=1123456)
fake_final = fake_wel[['Heading', 'Body', 'label']]
fake_final.dropna(inplace=True)

real_wel = real_wel.sample(n=len(satire), random_state=1123456)
real_final = real_wel[['Heading', 'Body', 'label']]
real_final.dropna(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [53]:
final_data = pd.concat([satire_final, fake_final, real_final])
final_data.dropna(inplace=True)

In [54]:
len(final_data[final_data['label'] == 'fake'])
#len(final_data[final_data['label'] == 'real'])
#len(final_data[final_data['label'] == 'satire'])

19929

In [55]:
final_data['Heading_Body'] = final_data[['Heading', 'Body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [56]:
len(final_data)

59787

In [57]:
#final_data.to_csv('unclean_data.csv', index=False)

In [58]:
import re
# Convert to lowercase
clean_final_data = final_data.applymap(lambda s: s.lower())

# Fix Contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
contraction_dict_2 = {"ain’t": "is not", "aren’t": "are not","can’t": "cannot", "’cause": "because", "could’ve": "could have", "couldn’t": "could not", "didn’t": "did not",  "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would","he’ll": "he will", "he’s": "he is", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is",  "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have","I’m": "I am", "I’ve": "I have", "i’d": "i would", "i’d’ve": "i would have", "i’ll": "i will",  "i’ll've": "i will have","i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have","it’s": "it is", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have","mightn’t": "might not","mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have","o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have","so’s": "so as", "this’s": "this is","that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would", "there’d’ve": "there would have", "there’s": "there is", "here’s": "here is","they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will", "they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are",  "what’s": "what is", "what’ve": "what have", "when’s": "when is", "when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have", "who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would","y’all’d’ve": "you all would have","y’all’re": "you all are","y’all’ve": "you all have","you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)
contractions_2, contractions_re_2 = _get_contractions(contraction_dict_2)

def replace_contractions(text):
    def replace_1(match):
        return contractions[match.group(0)]
    def replace_2(match):
        return contractions_2[match.group(0)]
    clean_1 = contractions_re.sub(replace_1, text)
    clean_2 = contractions_re_2.sub(replace_2, clean_1)
    return clean_2
clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(replace_contractions)
clean_final_data['Heading'] = clean_final_data['Heading'].apply(replace_contractions)
clean_final_data['Body'] = clean_final_data['Body'].apply(replace_contractions)

import string
# Remove Puncuation
def remove_punc(s):
    #punc_dict = {'!': ' ! ', '"': ' " ', '#': ' # ', '$': ' $ ', '%': ' % ', '&': ' & ', "'": " ' ", '(': ' ( ', ')': ' ) ', '*': ' * ', '+': ' + ', ',': ' , ', '-': ' - ', '.': ' . ', '/': ' / ', ':': ' : ', ';': ' ; ', '<': ' less than ', '=': ' equals ', '>': ' greater than ', '?': ' ? ', '@': ' @ ', '[': ' [ ', '\\': ' \\ ', ']': ' ] ', '^': ' ^ ', '_': ' _ ', '`': ' ` ', '{': ' { ', '|': ' | ', '}': ' } ', '~': ' ~ ', '—':' — '}
    punc_dict = {key: ' ' for key in string.punctuation}
    punc_dict['“'] = None
    punc_dict['—'] = ' '
    punc_dict['’'] = ' '
    table = str.maketrans(punc_dict)  # OR {key: None for key in string.punctuation}
    new_s = s.str.translate(table)
    return new_s

clean_final_data = clean_final_data.apply(remove_punc, axis=1)

news_name = "(babylon bee)|(onion)|(spoof)|(satire)|(reuters)|"
dates = "(2015)|(2016)|(2017)|(2018)|(2019)|(2021)|"
weird_extra = "(getelementbyid)|(cdata function)|(getelementsbytagname)|(createelement)"

pattern = news_name + dates + weird_extra

clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(lambda x: re.sub(pattern, " ", x))
clean_final_data['Heading'] = clean_final_data['Heading'].apply(lambda x: re.sub(pattern, " ", x))
clean_final_data['Body'] = clean_final_data['Body'].apply(lambda x: re.sub(pattern, " ", x))


In [59]:
# Remove extraneous HTML spaces
clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(lambda x: str(x).replace(u'\xa0', u' '))
clean_final_data['Heading'] = clean_final_data['Heading'].apply(lambda x: str(x).replace(u'\xa0', u' '))
clean_final_data['Body'] = clean_final_data['Body'].apply(lambda x: str(x).replace(u'\xa0', u' '))

In [60]:
# Remove extra white spaces between words
clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(lambda x: re.sub("\s\s+", " ", x))
clean_final_data['Heading'] = clean_final_data['Heading'].apply(lambda x: re.sub("\s\s+", " ", x))
clean_final_data['Body'] = clean_final_data['Body'].apply(lambda x: re.sub("\s\s+", " ", x))

In [61]:
fake = clean_final_data[clean_final_data.label == 'fake']

In [62]:
fake['Heading']

43310    african migrants in venice reject pasta demand...
11071    breaking obama says there were no scandals dur...
65502    dow nasdaq hit all time highs but the stock ma...
36427      one week and counting bottom fishing in america
64839    difficult” chelsea clinton causes high turnove...
                               ...                        
29912     video wow this racist statement from jeb bush...
63220    marco rubio is called irresponsible for buying...
2922     walt whitman s america through edward weston s...
70859     reporter reveals the real reason trump fired ...
43170     tennessee family values gop leader loses job ...
Name: Heading, Length: 19929, dtype: object

In [63]:
from collections import Counter
# Create Vocabulary
counter = Counter()
for comment in clean_final_data.Heading_Body:
    counter.update(comment.split())

counter

Counter({'here': 21040,
         'are': 136396,
         '10': 9599,
         'jokes': 558,
         'explained': 2724,
         'just': 46851,
         'in': 514955,
         'case': 11635,
         'you': 95259,
         'do': 52817,
         'not': 166779,
         'get': 27174,
         'them': 38256,
         'from': 116687,
         'the': 1521028,
         'atlantic': 828,
         'to': 778523,
         'pacific': 1546,
         'a': 623457,
         'lot': 8287,
         'of': 684478,
         'people': 62229,
         'all': 65303,
         'across': 8046,
         'country': 23234,
         'understand': 3880,
         'that': 375774,
         'is': 317013,
         'because': 30668,
         'we': 100067,
         'conservative': 6886,
         'christians': 1795,
         'bad': 6141,
         'at': 124064,
         'humor': 444,
         'nowhere': 602,
         'near': 4933,
         'as': 167080,
         'funny': 1211,
         'leftist': 1189,
         'humorists': 7,

In [64]:
clean_final_data

,Heading,Body,label,Heading_Body
0,here are 10 jokes explained just in case you d...,from the atlantic to the pacific a lot of peop...,satire,here are 10 jokes explained just in case you d...
1,pete buttigieg says he cannot come into work a...,washington d c with catastrophic labor shortag...,satire,pete buttigieg says he cannot come into work a...
2,instead of kryptonite new lgbtq superman will ...,burbank ca the brilliant and courageous writer...,satire,instead of kryptonite new lgbtq superman will ...
3,bernie retires as his vision of making the u s...,burlington vt bernie sanders has retired as a ...,satire,bernie retires as his vision of making the u s...
4,katie couric admits she edited interview to re...,u s katie couric has admitted to editing an in...,satire,katie couric admits she edited interview to re...
...,...,...,...,...
20759,u s flying bombers above disputed south china ...,the u s military confirmed a military exercise...,real,u s flying bombers above disputed south china ...
2664,battle over scalia s replacement already spill...,conservative and liberal advocacy groups are g...,real,battle over scalia s replacement already spill...
61642,trump administration to review goal of world w...,washington the trump administration is reviewi...,real,trump administration to review goal of world w...
21956,strong organization helps cruz dominate colora...,colorado springs colorado cnn ted cruz on satu...,real,strong organization helps cruz dominate colora...


In [65]:
#clean_final_data.to_csv('../data/clean_final_data_no_date.csv', index=False)

In [66]:
clean_final_data[clean_final_data['Heading_Body'].str.contains('getelementbyid')]

,Heading,Body,label,Heading_Body


In [67]:
clean_final_data['label'].unique()

array(['satire', 'fake', 'real'], dtype=object)